In [ ]:
!pip install git+https://github.com/google/tf-quant-finance
%matplotlib inline
import tensorflow as tf
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import datetime
import tf_quant_finance as tff
from scipy.stats import norm
#from polygon import RESTClient

pd.set_option('display.max_rows', 1000)

'''
FUNCTION: options_chain
  - Retrieves current call and put option data for input symbol

Inputs:
  - symbol (str) = Input singular ticker string

Output:
  - Pandas dataframe of option data for specified symbol. Includes contract symbol,
    strike, last price, bid, ask, volume, open interest, implied volatility,
    in the money (bool), expiration date,  time to expiry, and CALL (bool).
'''
def options_chain(symbol):
    tk = yf.Ticker(symbol)
    exps = tk.options
    options = pd.DataFrame()
    for e in exps:
        opt = tk.option_chain(e)
        opt = pd.DataFrame().append(opt.calls).append(opt.puts)
        opt['expirationDate'] = e
        options = options.append(opt, ignore_index=True)

    #add 1 day to get the correct expiration date
    options['expirationDate'] = pd.to_datetime(options['expirationDate']) + datetime.timedelta(days = 1)
    options['dte'] = (options['expirationDate'] - datetime.datetime.today()).dt.days
    options['CALL'] = options['contractSymbol'].str[4:].apply(lambda x: "C" in x)
    options[['bid', 'ask', 'strike']] = options[['bid', 'ask', 'strike']].apply(pd.to_numeric)
    options = options.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate'])
    return options

sym='NVDA'

C=options_chain(sym)

# Getting today's share price
tickerData = yf.Ticker(sym)
todayData = tickerData.history(period='1d')
S=todayData['Close'][0]
###print(f"Today's share price: \n {S} \n")

# Getting call option data for desired symbol
C=C[["strike","lastPrice","volume","CALL","dte","impliedVolatility"]]
C=C[(C["CALL"]==True)&(C["strike"]>S)&(C["volume"]>10)&(C["dte"]<np.max(C["dte"]/2))]
C=C[["strike","lastPrice","volume","dte","impliedVolatility"]]
l=C.shape[0]
###print(f"Call option data for {sym}: \n {C} \n")


'''
FUNCTION: SABR_data
  - Generates SABR parameters and option price for specified K&T with binary
    classification of bubble state

Inputs:
  - m (integer) = number of SABR option surfaces to generate
  - S (float) = current share price
  - C (pd dataframe) = Call option data with columns "strike" for strike price (K)
                       and "tde" for time to expiry (T).
  - plotting (0, 1) = choice whether to plot (1) or not plot (0) a comparison of
                      the real and SABR option prices for given K and T.

Outputs
  - Pandas dataframe of m generated sets of SABR parameters (alpha, sigma, rho) and SABR prices
    for each current call option (of specified K and T). A classification column ("lm") indicates
    if the generated data is/is not a bubble (local martingale).
  - [CONDITIONAL: plotting = 1] Prints a plot of the
'''
def SABR_data(m, S, C, plotting=0):
  #m is number of SABR option surface samples
  row_list = []

  # Generate m sets of SABR prices for randomised parameters (for NN training)
  for i in range(m):
    SABR=pd.DataFrame()
    # Randomised parameters
    SABR["alpha"]=np.random.uniform(0.15, 0.6)*np.ones(l)
    SABR["sigma"]=np.random.uniform(0.4, 6)*np.ones(l)
    SABR["rho"]=np.random.uniform(-0.8, 0.8)*np.ones(l) # split half half?
    # Specified parameters
    SABR["current"]=S*np.ones(l)
    SABR["strike"]=C["strike"].values
    SABR["exp"]=C["dte"].values/365. #*np.ones(n)#np.random.uniform(0,5,n)
    # Calculate SABR option price
    SABR["price"]=tff.models.sabr.approximations.european_option_price(
        strikes=SABR["strike"],
        expiries=SABR["exp"],
        forwards=SABR["current"],
        is_call_options=True,
        alpha=SABR["alpha"],
        beta=1,
        volvol=SABR["sigma"],
        rho=SABR["rho"],
        dtype=tf.float64)

    if plotting==1:
        fig2 = plt.figure(figsize=(10,10))
        ax2 = fig2.add_subplot(111, projection='3d')
        # Compare last observed price with SABR price.
        ax2.plot_trisurf(C["strike"].values, C["dte"].values, C["lastPrice"].values, label='Market prices', color='#1f77b4') #blue
        ax2.plot_trisurf(C["strike"].values, C["dte"].values, SABR["price"], label='SABR prices', color='#ff7f0e') #orange
        ax2.set_xlabel('Strike price')
        ax2.set_ylabel('Time to expiry')
        ax2.set_zlabel('Call price')
        plt.tighten_layout()
        plt.show()

    row_list.append({"alpha":SABR["alpha"][0], "sigma":SABR["sigma"][0],
                     "rho":SABR["rho"][0], "prices":SABR["price"].values})

  X=pd.DataFrame(row_list)
  # lm class 1 for bubble and 0 for no bubble.
  X["lm"]=np.where(X['rho']>0, 1, 0)

  return X

yeet=10000
X=SABR_data(yeet, S, C, plotting=0)#["prices"][0])

fig2= plt.figure(figsize=(10,10))
ax2 = fig2.add_subplot(111, projection='3d')
ax2.plot_trisurf(C["strike"].values, C["dte"].values, C["lastPrice"].values, label='Market prices', color='#1f77b4') #blue
ax2.plot_trisurf(C["strike"].values, C["dte"].values, SABR_data(10, S, C)["prices"][0], label='SABR prices', color='#ff7f0e') #orange
ax2.set_xlabel('Strike price')
ax2.set_ylabel('Time to expiry')
ax2.set_zlabel('Call price')
plt.tight_layout()
plt.show()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/google/tf-quant-finance to /tmp/pip-req-build-1h965d52
  Running command git clone --filter=blob:none --quiet https://github.com/google/tf-quant-finance /tmp/pip-req-build-1h965d52
  Resolved https://github.com/google/tf-quant-finance to commit 7e603920700660e28a05c42764cae1922ed2e40b
  Preparing metadata (setup.py) ... done
  Created wheel for tf-quant-finance: filename=tf_quant_finance-0.0.1.dev34-py3-none-any.whl size=1017039 sha256=f0c1fe49c9f2bb46bacc63fc6185a9e72a5105a2996ebcc4626e19699c765c3b
  Stored in directory: /tmp/pip-ephem-wheel-cache-crck0b2r/wheels/4b/6d/30/ed70de2199222e09f436b48b0736b821fc2efdfe7c497949bd
Successfully built tf-quant-finance


In [ ]:
### // Neural Network Section // ###
# Use K & T values from current options as input --> should use others as well for generation (doesnt have to be the same as that of the current options market for generative training)
'''
Paper suggests:
  - 3000 samples, 2000 training and 1000 test
  - half being martingales (rho U(-0.8, 0)) and half strict local martingales (rho U(0, 0.8))

"We take 61 maturities equally spaced from T1 = 2 to Tl = 5 and, as in Section 3.1, two
groups of 100 equally spaced strikes: one from K1 = 1 to Km = 3.5 and one from K1 = 3
to Km = 5.5. We consider three different classes of processes for the underlying, in all cases
with initial value X0 = 2:"

"We choose ntrain=2000, ntest = 1000, a = 0.5, A = 3, D = 0.2, σ = 0.2, X0 = 2"
'''

X=X[["prices","lm"]]
lm=X["lm"]
print(X.shape)
Y=X["prices"].to_numpy()
Y=np.vstack(Y)
print(Y.shape)


l=C.shape[0]
n = len(Y[0])

#print(np.array([C["lastPrice"].values]))
#print(model.predict(np.array([C["lastPrice"].values]))[0])

model=tf.keras.Sequential([
    tf.keras.layers.Dropout(.05),
    tf.keras.layers.Dense(40, activation='relu'),
    tf.keras.layers.Dropout(.1),
    tf.keras.layers.Dense(40, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

model.fit(Y,lm,epochs=40)
print(np.array([C["lastPrice"].values]))
print(model.predict(np.array([C["lastPrice"].values]))[0])


In [ ]:
test_stocks = ['AAPL', 'NVDA', 'AMZN', 'NFLX', 'TSLA', ]